In [1]:
import pickle
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.optimizers import Adam
from keras.models import Model, Sequential
from keras.layers import Dense, Activation, Dropout, ReLU

from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import pearsonr, spearmanr


2024-09-12 14:13:45.029609: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-12 14:13:45.079809: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-12 14:13:45.924921: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
  except RuntimeError as e:
    print(e)

2024-09-12 14:13:47.273396: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-12 14:13:47.323789: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-12 14:13:47.324128: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [3]:
adj_matrix = pd.read_csv(("/home/wootaek/taek_HDD/deepgoCMAP/data/adj_matrix.csv"),index_col=0)
first_matrix_connection = pd.read_csv(("/home/wootaek/deepgoCMAP-drug/data_preprocess/connection_matrix/first_GO_matrix_cmap_12014x1574_deg랑유전자맞춤.csv"),index_col=0)
csv_go = pd.read_csv(("/home/wootaek/taek_HDD/deepgoCMAP/data/go_level.csv"),index_col=0)

In [4]:
with open('/home/wootaek/deepgoCMAP-drug/data_preprocess/lv5_cp_24h_CTRP_train_x_32621x12014_240131.pkl', 'rb') as file:
    X_train = pickle.load(file)
with open('/home/wootaek/deepgoCMAP-drug/data_preprocess/lv5_cp_24h_CTRP_valid_x_4078x12014_240131.pkl', 'rb') as file:
    X_val = pickle.load(file)
with open('/home/wootaek/deepgoCMAP-drug/data_preprocess/lv5_cp_24h_CTRP_test_x_4078x12014_240131.pkl', 'rb') as file:
    X_test = pickle.load(file)
with open('/home/wootaek/deepgoCMAP-drug/data_preprocess/lv5_cp_24h_CTRP_train_y+info_240131.pkl', 'rb') as file:
    y_train = pickle.load(file)
with open('/home/wootaek/deepgoCMAP-drug/data_preprocess/lv5_cp_24h_CTRP_valid_y+info_240131.pkl', 'rb') as file:
    y_val = pickle.load(file)
with open('/home/wootaek/deepgoCMAP-drug/data_preprocess/lv5_cp_24h_CTRP_test_y+info_240131.pkl', 'rb') as file:
    y_test = pickle.load(file)

In [5]:
X_train = X_train.astype(float).values
X_val = X_val.astype(float).values
X_test = X_test.astype(float).values

In [6]:
y_train = y_train['cell_viability'].astype(float).values
y_val = y_val['cell_viability'].astype(float).values
y_test = y_test['cell_viability'].astype(float).values

In [7]:
def custom_loss_with_l2_reg(lambda_value):
    @tf.autograph.experimental.do_not_convert
    def loss(y_true, y_pred):
        mse_loss = tf.keras.losses.mean_squared_error(y_true, y_pred)
        l2_reg = 0
        for i, w in enumerate(model.trainable_weights):
            if i % 2 == 0:
                l2_reg += tf.nn.l2_loss(w *(1 - connection_matrix[i//2]))
            else:
                l2_reg += tf.nn.l2_loss(w)
        
        return mse_loss + lambda_value * l2_reg
    return loss

In [8]:
connection_matrix = []
connection_matrix.append(np.array(first_matrix_connection.values,dtype=np.float32))
connection_matrix.append(np.array(adj_matrix.loc[csv_go[str(7)].loc[lambda x: x==1].index,csv_go[str(6)].loc[lambda x: x==1].index].values,dtype=np.float32))
connection_matrix.append(np.array(adj_matrix.loc[csv_go[str(6)].loc[lambda x: x==1].index,csv_go[str(5)].loc[lambda x: x==1].index].values,dtype=np.float32))
connection_matrix.append(np.array(adj_matrix.loc[csv_go[str(5)].loc[lambda x: x==1].index,csv_go[str(4)].loc[lambda x: x==1].index].values,dtype=np.float32))
connection_matrix.append(np.ones((515,1),dtype=np.float32))

In [9]:
def create_model():
    model = Sequential([
        Dense(1574, activation=ReLU(), input_dim=12014),
        Dropout(0.3),
        Dense(1386, activation=ReLU()),
        Dropout(0.3),
        Dense(951, activation=ReLU()),
        Dropout(0.3),
        Dense(515, activation=ReLU()),
        Dropout(0.3),
        Dense(1, activation='linear')
    ])
    return model

In [10]:
def calculate_metrics(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    pearson_corr, _ = pearsonr(y_true.flatten(), y_pred.flatten())
    spearman_corr, _ = spearmanr(y_true.flatten(), y_pred.flatten())
    
    return mse, rmse, r2, pearson_corr, spearman_corr

def train_model(model, X_train, y_train, X_valid, y_valid, epochs, learning_rate, lambda_value, batch_size):
    tf.random.set_seed(33)
    opt = Adam(learning_rate=learning_rate)
    loss_fn = custom_loss_with_l2_reg(lambda_value)
    model.compile(optimizer=opt, loss=loss_fn, metrics=['mean_squared_error'])

    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_valid, y_valid))
    return model, history

def evaluate_model(model, X_test, y_test, X_val, y_val, history, lambda_value, epochs, learning_rate, batch_size):
    best_epoch = np.argmin(history.history['val_loss'])
    best_loss = history.history['val_loss'][best_epoch]
    best_mse = history.history['val_mean_squared_error'][best_epoch]

    y_pred_test = model.predict(X_test)
    y_pred_val = model.predict(X_val)

    val_metrics = calculate_metrics(y_val, y_pred_val)
    test_metrics = calculate_metrics(y_test, y_pred_test)

    results_dict = {
        "Lambda Value": lambda_value,
        "Best Epoch": best_epoch,
        "Best Validation Loss": best_loss,
        "Best Validation Mean Squared Error": best_mse,
        "Validation Mean Squared Error": val_metrics[0],
        "Validation Root Mean Squared Error": val_metrics[1],
        "Validation R-squared": val_metrics[2],
        "Validation Pearson Correlation": val_metrics[3],
        "Validation Spearman Correlation": val_metrics[4],
        "Test Mean Squared Error": test_metrics[0],
        "Test Root Mean Squared Error": test_metrics[1],
        "Test R-squared": test_metrics[2],
        "Test Pearson Correlation": test_metrics[3],
        "Test Spearman Correlation": test_metrics[4]
    }

    results_df = pd.DataFrame([results_dict])

    return model, results_df

In [12]:
epochs = 200
learning_rate = 0.00001
batch_size = 128  
lambda_value = 0.0001  

model = create_model()
model, history = train_model(model, X_train, y_train, X_val, y_val, 
                             epochs=epochs, learning_rate=learning_rate, 
                             lambda_value=lambda_value, batch_size=batch_size)

model, results_df = evaluate_model(model, X_test, y_test, X_val, y_val, 
                                   history, epochs=epochs, learning_rate=learning_rate, 
                                   lambda_value=lambda_value, batch_size=batch_size)

print("Model evaluation complete.")

Epoch 1/200
255/255 [==============================] - 14s 27ms/step - loss: 1.0153 - mean_squared_error: 0.7130 - val_loss: 0.5302 - val_mean_squared_error: 0.2279
Epoch 2/200
255/255 [==============================] - 1s 5ms/step - loss: 0.6584 - mean_squared_error: 0.3561 - val_loss: 0.5319 - val_mean_squared_error: 0.2297
Epoch 3/200
255/255 [==============================] - 1s 5ms/step - loss: 0.5535 - mean_squared_error: 0.2514 - val_loss: 0.5284 - val_mean_squared_error: 0.2264
Epoch 4/200
255/255 [==============================] - 1s 6ms/step - loss: 0.5067 - mean_squared_error: 0.2047 - val_loss: 0.5314 - val_mean_squared_error: 0.2294
Epoch 5/200
255/255 [==============================] - 1s 5ms/step - loss: 0.4715 - mean_squared_error: 0.1696 - val_loss: 0.5261 - val_mean_squared_error: 0.2243
Epoch 6/200
255/255 [==============================] - 1s 5ms/step - loss: 0.4536 - mean_squared_error: 0.1518 - val_loss: 0.5283 - val_mean_squared_error: 0.2266
Epoch 7/200
255/255 

In [13]:
results_df

,Lambda Value,Best Epoch,Best Validation Loss,Best Validation Mean Squared Error,Validation Mean Squared Error,Validation Root Mean Squared Error,Validation R-squared,Validation Pearson Correlation,Validation Spearman Correlation,Test Mean Squared Error,Test Root Mean Squared Error,Test R-squared,Test Pearson Correlation,Test Spearman Correlation
0,0.0001,199,0.049858,0.020577,0.020577,0.143447,0.746379,0.863999,0.650989,0.022401,0.14967,0.732881,0.856506,0.651551
